In [79]:
import pandas as pd
import csv

df = pd.read_csv('filteredDataWithPosition.csv')


In [80]:
image_dict = {}

for index, row in df.iterrows():
    image_id = row['image_id']
    category_name = row['categoryName']
    position_name = row['positionName']
    
    if image_id not in image_dict:
        image_dict[image_id] = []
    
    image_dict[image_id].append((category_name, position_name))


In [81]:
ct = 0
for k, v in image_dict.items():
    ct += 1
    print(k, v)
    if ct == 4:
        break


print(len(image_dict))

289343 [('dog', 'middleRight'), ('person', 'middleCenter'), ('bench', 'bottomLeft'), ('bicycle', 'middleCenter')]
61471 [('dog', 'bottomCenter'), ('bottle', 'topLeft'), ('toilet', 'topCenter')]
472375 [('dog', 'middleCenter'), ('motorcycle', 'middleCenter')]
520301 [('dog', 'middleCenter')]
4109


In [82]:
def constructQuestionAndAnswer(obj1, pos1, variationNum):
    questionList = [
        f"Where is the {obj1} located in the image?",
        f"What is the position of the {obj1} in the image?",
        f"Can you tell me where the {obj1} is in the picture?",
        f"Describe the location of the {obj1} in the image.",
        f"Locate the {obj1} in the image.",
        f"Identify the position of the {obj1} in the picture.",
        f"Where can I find the {obj1} in the image?",
        f"Point out the location of the {obj1}.",
        f"Tell me about the placement of the {obj1} in the image.",
        f"Give me the whereabouts of the {obj1} in the picture.",
        f"Specify the position of the {obj1} in the image.",
        f"Elaborate on the whereabouts of the {obj1} in the picture.",
        f"In which part of the image is the {obj1} situated?",
        f"Pinpoint the location of the {obj1} in the image.",
        f"Describe where the {obj1} is placed in the picture.",
        f"Locate the the {obj1} within the image.",
        f"Indicate the position of the {obj1} in the picture.",
        f"Describe the spatial location of the {obj1} in the image.",
        f"Identify the location of the {obj1} in the picture.",
        f"Specify the exact position of the {obj1} in the image.",
        f"Tell me about the spatial position of the {obj1} in the picture.",
        f"What quadrant is the {obj1} in the image?",
        f"Which area of the image is the {obj1} located in?",
        f"Tell me the spot where the {obj1} is placed in the picture.",
        f"Provide information about the location of the {obj1} in the image.",
        f"Identify the general area where the {obj1} is located in the picture.",
        f"Elaborate on the general whereabouts of the {obj1} in the picture.",
        f"Tell me about the placement of the {obj1} in the image."
        ]

    
    variationNum = variationNum % len(questionList)
    answer = f"The {obj1} is located at the {pos1} of the image."
    return questionList[variationNum], answer

def combine2QuestionsAndAnswers(q1, a1, q2, a2, obj1, obj2):
    combinations = []
    fixedAnswer = a1[:-1] + ' and ' + a2[0].lower() + a2[1:]
    combinations.append( (q1[:-1] + ' and ' + q2[0].lower() + q2[1:], fixedAnswer))
    q1 = q1.replace(obj1, obj1 + " and the " + obj2)
    q1 = q1.replace(" is ", " are ")
    combinations.append( (q1, fixedAnswer))
    return combinations


        

In [83]:
questionsAndAnswers = []
varNum = 0
for k, v in image_dict.items():
    if len(v) == 1:
        obj1, pos1 = v[0]
        q, a = constructQuestionAndAnswer(obj1, pos1, varNum)
        questionsAndAnswers.append((k, q, a))
        varNum += 1
    else:
        for i in range(len(v)-1):
            obj1, pos1 = v[i]
            obj2, pos2 = v[i+1]
            variationsNeeded = 3
            for j in range(variationsNeeded):
                q1, a1 = constructQuestionAndAnswer(obj1, pos1, varNum)
                q2, a2 = constructQuestionAndAnswer(obj2, pos2, varNum%11 + varNum%7)
                questionsAndAnswers.append((k, q1, a1))
                questionsAndAnswers.append((k, q2, a2))
                qAndAs = combine2QuestionsAndAnswers(q1, a1, q2, a2, obj1, obj2)
                for q, a in qAndAs:
                    questionsAndAnswers.append((k, q, a))
                varNum += 1
            
print(f"{len(questionsAndAnswers)} questions and answers generated.")

54874 questions and answers generated.


In [84]:
for i in range(len(questionsAndAnswers)):
    questionsAndAnswers[i] = (i,) + questionsAndAnswers[i]

questionsAndAnswers[:10]

[(0,
  289343,
  'Where is the dog located in the image?',
  'The dog is located at the middleRight of the image.'),
 (1,
  289343,
  'Where is the person located in the image?',
  'The person is located at the middleCenter of the image.'),
 (2,
  289343,
  'Where is the dog located in the image and where is the person located in the image?',
  'The dog is located at the middleRight of the image and the person is located at the middleCenter of the image.'),
 (3,
  289343,
  'Where are the dog and the person located in the image?',
  'The dog is located at the middleRight of the image and the person is located at the middleCenter of the image.'),
 (4,
  289343,
  'What is the position of the dog in the image?',
  'The dog is located at the middleRight of the image.'),
 (5,
  289343,
  'Can you tell me where the person is in the picture?',
  'The person is located at the middleCenter of the image.'),
 (6,
  289343,
  'What is the position of the dog in the image and can you tell me where

In [85]:

filename = 'nlp_coco_questions_and_answers.csv'
delimiter = '|'

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file, delimiter=delimiter)
    writer.writerow(['id', 'image_id', 'question', 'answer'])
    writer.writerows(questionsAndAnswers)

print(f"Questions and answers saved to {filename}.")


Questions and answers saved to nlp_coco_questions_and_answers.csv.
